In [7]:
!pip3 install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 11.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 9.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [17]:
import os
import numpy as np
import tensorflow as tf

from keras.utils import load_img, img_to_array
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Flatten, Dense, Dropout
from keras import Model, callbacks

In [2]:
def init_vgg16():

    # Download the model with weights pre-trained using ImageNet database
    vgg16 = VGG16(weights='imagenet', 
                  include_top=False,
                  input_shape=(224,224,3))
    
    
    # Freeze layers for training
    for layer in vgg16.layers:
        layer.trainable = False

    # Create a new 'top' of the model of fully-connected layers
    top_model = vgg16.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu', name="top_dense_1")(top_model)
    top_model = Dropout(0.2)(top_model)
    top_model = Dense(512, activation='relu', name="top_dense_2")(top_model)
    top_model = Dropout(0.2)(top_model)
    top_model = Dense(256, activation='relu', name="top_dense_3")(top_model)
    top_model = Dropout(0.2)(top_model)
    top_model = Dense(128, activation='relu', name="top_dense_4")(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(1, activation='softmax', name="output" )(top_model)
    
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=vgg16.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer='adam', 
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [3]:
def load_and_preprocess_img(path: str):

    img = load_img(path, target_size=(224, 224))
    ary = img_to_array(img)
    ary = np.expand_dims(ary, axis=0)
    ary = preprocess_input(ary)
    return ary[0]


In [4]:
from os import listdir

def load_data(path: str, label: int):

    X = []
    y = []

    for filename in listdir(path):
        x = load_and_preprocess_img(path + filename)
        X.append(x)
        y.append(label)

    return X, y


In [5]:
FILEPATH_POS = "./out_pos/"
FILEPATH_NEG = "./out_neg/"

X, y = load_data(FILEPATH_POS, 1)
X_neg, y_neg = load_data(FILEPATH_NEG, 0)

X.extend(X_neg)
y.extend(y_neg)

X = np.array(X)
y = np.array(y)

print(f"X: {X.shape}")
print(f"y: {y.shape}")


X: (6767, 224, 224, 3)
y: (6767,)


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

In [9]:
model = init_vgg16()
model.summary()

2023-04-19 07:58:25.310853: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-19 07:58:25.313792: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-19 07:58:25.313918: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

58889256/58889256 [==============================] - 7s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [11]:
model.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
142/142 [==============================] - 14s 101ms/step - loss: 0.5306 - accuracy: 0.9173
Epoch 2/100
142/142 [==============================] - 14s 101ms/step - loss: 0.3191 - accuracy: 0.9173
Epoch 3/100
142/142 [==============================] - 14s 101ms/step - loss: 0.1484 - accuracy: 0.9173
Epoch 4/100
142/142 [==============================] - 14s 101ms/step - loss: 0.0983 - accuracy: 0.9173
Epoch 5/100
142/142 [==============================] - 14s 101ms/step - loss: 0.0839 - accuracy: 0.9173
Epoch 6/100
142/142 [==============================] - 14s 101ms/step - loss: 0.0574 - accuracy: 0.9173
Epoch 7/100
142/142 [==============================] - 14s 101ms/step - loss: 0.0301 - accuracy: 0.9173
Epoch 8/100
142/142 [==============================] - 14s 101ms/step - loss: 0.0536 - accuracy: 0.9173
Epoch 9/100
142/142 [==============================] - 14s 101ms/step - loss: 0.0558 - accuracy: 0.9173
Epoch 10/100
142/142 [==============================] - 14s 101m

142/142 [==============================] - 14s 99ms/step - loss: 0.0134 - accuracy: 0.9173
Epoch 80/100
142/142 [==============================] - 14s 101ms/step - loss: 0.0128 - accuracy: 0.9173
Epoch 81/100
142/142 [==============================] - 14s 102ms/step - loss: 0.0530 - accuracy: 0.9173
Epoch 82/100
142/142 [==============================] - 15s 102ms/step - loss: 0.1380 - accuracy: 0.9173
Epoch 83/100
142/142 [==============================] - 14s 102ms/step - loss: 0.0822 - accuracy: 0.9173
Epoch 84/100
142/142 [==============================] - 14s 102ms/step - loss: 0.1818 - accuracy: 0.9173
Epoch 85/100
142/142 [==============================] - 14s 101ms/step - loss: 0.0488 - accuracy: 0.9173
Epoch 86/100
142/142 [==============================] - 14s 102ms/step - loss: 0.2283 - accuracy: 0.9173
Epoch 87/100
142/142 [==============================] - 14s 102ms/step - loss: 0.1648 - accuracy: 0.9173
Epoch 88/100
142/142 [==============================] - 14s 101ms/ste

In [13]:
model.evaluate(X_test, y_test, batch_size=32)

70/70 [==============================] - 6s 92ms/step - loss: 9.1525 - accuracy: 0.9114


[9.152471542358398, 0.9113697409629822]

In [ ]:
# Checkpoint during training
checkpoint_path = "classifier_chkpts/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                        save_weights_only=True,
                                        verbose=1)

model.fit(X_train,
          y_train,
          batch_size=16,
          epochs=200,
          validation_data=(X_test, y_test),
          callbacks=[cp_callback])

Create model
Epoch 1/200
284/284 [==============================] - ETA: 0s - loss: 3.5205 - accuracy: 0.9173
Epoch 1: saving model to classifier_chkpts/cp.ckpt
284/284 [==============================] - 29s 96ms/step - loss: 3.5205 - accuracy: 0.9173 - val_loss: 0.4245 - val_accuracy: 0.9114
Epoch 2/200
283/284 [============================>.] - ETA: 0s - loss: 0.5745 - accuracy: 0.9172
Epoch 2: saving model to classifier_chkpts/cp.ckpt
284/284 [==============================] - 26s 92ms/step - loss: 0.5739 - accuracy: 0.9173 - val_loss: 0.5072 - val_accuracy: 0.9114
Epoch 3/200
283/284 [============================>.] - ETA: 0s - loss: 0.2630 - accuracy: 0.9174
Epoch 3: saving model to classifier_chkpts/cp.ckpt
284/284 [==============================] - 25s 88ms/step - loss: 0.2628 - accuracy: 0.9173 - val_loss: 0.2825 - val_accuracy: 0.9114
Epoch 4/200
283/284 [============================>.] - ETA: 0s - loss: 0.2680 - accuracy: 0.9172
Epoch 4: saving model to classifier_chkpts/cp.c

Epoch 30/200
283/284 [============================>.] - ETA: 0s - loss: 0.1330 - accuracy: 0.9172
Epoch 30: saving model to classifier_chkpts/cp.ckpt
284/284 [==============================] - 25s 87ms/step - loss: 0.1329 - accuracy: 0.9173 - val_loss: 1.5605 - val_accuracy: 0.9114
Epoch 31/200
283/284 [============================>.] - ETA: 0s - loss: 0.0847 - accuracy: 0.9172
Epoch 31: saving model to classifier_chkpts/cp.ckpt
284/284 [==============================] - 25s 87ms/step - loss: 0.0846 - accuracy: 0.9173 - val_loss: 0.6040 - val_accuracy: 0.9114
Epoch 32/200
 47/284 [===>..........................] - ETA: 14s - loss: 0.0807 - accuracy: 0.9215